In [ ]:
import numpy as np
import os
import random
import wandb
import torch
import logging
from arguments import parser
from torch.utils.data import DataLoader

from train import refinement_run
from datasets import create_dataset
from log import setup_default_logging

from accelerate import Accelerator
from omegaconf import OmegaConf

from torch.distributed import get_rank
torch.autograd.set_detect_anomaly(True)

default_setting = './configs/benchmark/default_setting.yaml'
cfg = parser(jupyter=True, default_setting = default_setting)



accelerator = Accelerator(
    mixed_precision             = cfg.TRAIN.mixed_precision
)

# load dataset
trainset, validset, testset = create_dataset(
    dataset_name  = cfg.DATASET.dataset_name,
    datadir       = cfg.DATASET.datadir,
    class_name    = cfg.DATASET.class_name,
    img_size      = cfg.DATASET.img_size,
    mean          = cfg.DATASET.mean,
    std           = cfg.DATASET.std,
    aug_info      = cfg.DATASET.aug_info,
    anomaly_ratio = 0.1,
    **cfg.DATASET.get('params',{})
)

# make save directory
savedir = os.path.join(
                            cfg.DEFAULT.savedir,
                            cfg.DATASET.dataset_name,
                            cfg.DATASET.class_name
                        )

exp_name         = cfg.DEFAULT.exp_name

method            = cfg.MODEL.method
model_name        = cfg.MODEL.model_name
model_params      = cfg.MODEL.get('params',{})

trainset         = trainset
validset         = validset
testset          = testset

batch_size       = cfg.DATASET.batch_size
test_batch_size  = cfg.DATASET.test_batch_size
num_workers      = cfg.DATASET.num_workers

opt_name         = cfg.OPTIMIZER.opt_name
lr               = cfg.OPTIMIZER.lr
opt_params       = cfg.OPTIMIZER.get('params',{})

epochs           = cfg.TRAIN.epochs
log_interval     = cfg.TRAIN.log_interval
use_wandb        = cfg.TRAIN.wandb.use

savedir          = savedir
seed             = cfg.DEFAULT.seed
accelerator      = accelerator
cfg              = cfg

model = __import__('models').__dict__[method](
    model_name = model_name,
    **model_params
    )    

# define train dataloader
trainloader = DataLoader(
    dataset     = trainset,
    batch_size  = batch_size,
    num_workers = num_workers
)

validloader = DataLoader(
    dataset     = validset,
    batch_size  = test_batch_size,
    num_workers = num_workers
)

# define test dataloader
testloader = DataLoader(
    dataset     = testset,
    batch_size  = test_batch_size,
    shuffle     = False,
    num_workers = num_workers
)

optimizer = __import__('torch.optim', fromlist='optim').__dict__[opt_name](model.parameters(), lr=lr, **opt_params)

scheduler = None 

model, optimizer, trainloader, testloader, scheduler = accelerator.prepare(
            model, optimizer, trainloader, testloader, scheduler
        )

r = 0 

from sklearn.metrics import roc_auc_score,auc 
from scipy.ndimage import gaussian_filter
import numpy as np 
from numpy import ndarray
import pandas as pd 
import torch.nn.functional as F 
import torch 
from skimage import measure
from statistics import mean

def compute_pro(masks: np.ndarray, amaps: np.ndarray, num_th: int = 200) -> None:

    """Compute the area under the curve of per-region overlaping (PRO) and 0 to 0.3 FPR
    Args:
        category (str): Category of product
        masks (ndarray): All binary masks in test. masks.shape -> (num_test_data, h, w)
        amaps (ndarray): All anomaly maps in test. amaps.shape -> (num_test_data, h, w)
        num_th (int, optional): Number of thresholds
    """

    assert isinstance(amaps, ndarray), "type(amaps) must be ndarray"
    assert isinstance(masks, ndarray), "type(masks) must be ndarray"
    assert amaps.ndim == 3, "amaps.ndim must be 3 (num_test_data, h, w)"
    assert masks.ndim == 3, "masks.ndim must be 3 (num_test_data, h, w)"
    assert amaps.shape == masks.shape, "amaps.shape and masks.shape must be same"
    #assert set(masks.flatten()) == {0, 1}, "set(masks.flatten()) must be {0, 1}"
    assert isinstance(num_th, int), "type(num_th) must be int"

    df = pd.DataFrame([], columns=["pro", "fpr", "threshold"])
    binary_amaps = np.zeros_like(amaps, dtype=np.bool)

    min_th = amaps.min()
    max_th = amaps.max()
    delta = (max_th - min_th) / num_th

    for i,th in enumerate(np.arange(min_th, max_th, delta)):
        binary_amaps[amaps <= th] = 0
        binary_amaps[amaps > th] = 1

        pros = []
        for binary_amap, mask in zip(binary_amaps, masks):
            for region in measure.regionprops(measure.label(mask)):
                axes0_ids = region.coords[:, 0]
                axes1_ids = region.coords[:, 1]
                tp_pixels = binary_amap[axes0_ids, axes1_ids].sum()
                pros.append(tp_pixels / region.area)

        inverse_masks = 1 - masks
        fp_pixels = np.logical_and(inverse_masks, binary_amaps).sum()
        fpr = fp_pixels / inverse_masks.sum()

        
        df.loc[i,:] = [mean(pros),fpr,th]
    # Normalize FPR from 0 ~ 1 to 0 ~ 0.3
    df = df[df["fpr"] < 0.3]
    df["fpr"] = df["fpr"] / df["fpr"].max()

    pro_auc = auc(df["fpr"], df["pro"])
    return pro_auc


In [3]:
from train import get_score_map

dataloader = testloader 
true_labels = [] 
score_list = [] 
true_gts = []

model.eval()
for idx, (images, labels, gts) in enumerate(dataloader):
    # predict
    t_f, s_f = model._forward(images)
    break 

/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:556: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 8, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [4]:
for t, s in zip(t_f, s_f):
    break 

In [20]:
f_t_I_k = t[0]

In [23]:
f_t_I_k_ij = f_t_I_k[:,0,0]